In [1]:
!pip install transformers

     ---------------------------------------- 7.1/7.1 MB 9.7 MB/s eta 0:00:00
     ------------------------------------- 224.5/224.5 kB 13.4 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 10.5 MB/s eta 0:00:00


In [3]:
!pip install tensorflow

     -------------------------------------- 272.8/272.8 MB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 9.8 MB/s eta 0:00:00
     --------------------------------------- 14.7/14.7 MB 10.6 MB/s eta 0:00:00
     -------------------------------------- 440.7/440.7 kB 9.2 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     ---------------------------------------- 5.6/5.6 MB 10.9 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ---------------------------------------- 4.1/4.1 MB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 11.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finish

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.5 which is incompatible.


In [6]:
!pip install torch

     -------------------------------------- 172.4/172.4 MB 4.7 MB/s eta 0:00:00


In [1]:
from PIL import Image
import requests
from transformers import (BlipProcessor, BlipForConditionalGeneration, 
                          AutoTokenizer, AutoModelForSeq2SeqLM)

In [40]:
blip_processor = BlipProcessor.from_pretrained("blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("blip-image-captioning-base")

In [28]:
def generate_desc(img_url,text,url):
    if url==True:
        raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
    else:
        raw_image = Image.open(img_url).convert('RGB')
    
    if text!='':
        # conditional image captioning
        inputs = blip_processor(raw_image, text, return_tensors="pt")
        out = blip_model.generate(**inputs)
        return(blip_processor.decode(out[0], skip_special_tokens=True))
    else:
        # unconditional image captioning
        inputs = blip_processor(raw_image, return_tensors="pt")
        out = blip_model.generate(**inputs)
        return(blip_processor.decode(out[0], skip_special_tokens=True))

In [41]:
t5_tokenizer = AutoTokenizer.from_pretrained('text-to-social-media-captions')
t5_model = AutoModelForSeq2SeqLM.from_pretrained('text-to-social-media-captions')

In [30]:
def generate_caption(text = """a woman playing soccer"""):
    
    inputs = ["captionize: " + text]
    
    inputs = t5_tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
    output = t5_model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
    decoded_output = t5_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    
    return decoded_output

In [29]:
def generate_caption_for_img(img_link='https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg',text = "",url=False,img_show=True):
    
    img_desc = generate_desc(img_link,text,url)
    if img_show==True:
        if url==True:
            Image.open(requests.get(img_link, stream=True).raw).show()
        else:
            Image.open(img_link).show()

    print('Image Description:',img_desc)
    
    caption = generate_caption(img_desc)
    print('Caption:',caption)

In [25]:
generate_caption_for_img(url=True)

Image Description: a woman sitting on the beach with her dog
Caption: "Unleashing the magic of nature's bounty."


In [27]:
img_link = fr"C:\Users\sgp\Downloads\Image1.png".replace('\\','/')
generate_caption_for_img(img_link)

False
Image Description: a man running on a soccer field
Caption: "Successful moments with a man on the soccer field."


In [48]:
%%writefile app.py
import streamlit as st
from PIL import Image
import requests
from transformers import (BlipProcessor, BlipForConditionalGeneration, 
                          AutoTokenizer, AutoModelForSeq2SeqLM)

#Load the models
@st.cache_resource
def get_models():
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    t5_tokenizer = AutoTokenizer.from_pretrained('prasanthsagirala/text-to-social-media-captions')
    t5_model = AutoModelForSeq2SeqLM.from_pretrained('prasanthsagirala/text-to-social-media-captions')
    return blip_processor,blip_model,t5_tokenizer,t5_model

blip_processor,blip_model,t5_tokenizer,t5_model = get_models()

#get the description 
def generate_desc(img_url):
    raw_image = Image.open(img_url).convert('RGB')
    inputs = blip_processor(raw_image, return_tensors="pt")
    out = blip_model.generate(**inputs)
    return(blip_processor.decode(out[0], skip_special_tokens=True))
    
def generate_caption(text):
    inputs = ["captionize: " + text]
    inputs = t5_tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")
    output = t5_model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
    decoded_output = t5_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return decoded_output

def generate_caption_for_img(img_link):    
    img_desc = generate_desc(img_link)
    caption = generate_caption(img_desc)
    return img_desc,caption

#User Input
def main():
    st.title('Caption Generator from Image')
    image_file = st.file_uploader("Upload Images", type=["png","jpg","jpeg"])
    if image_file!=None:
        st.image(Image.open(image_file),width=250)
        if st.button('Captionize'):
            caption = generate_caption_for_img(image_file)
            st.write('Image Description: ',caption[0])
            st.write('Caption: ',caption[1])

if __name__ == '__main__':
    main()

Overwriting app.py
